In [1]:
import MBTI_prediction_by_3ways_jp

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

In [3]:
df = pd.read_csv('C:/Users/JenMing/Desktop/MBTI/LSTM/mbti_to_LSTM_DF.csv')
df.head()

,type,posts
0,INFJ,"['INFP', 'INFP', 'INFJ', 'ENFP', 'ISTP', 'INTP..."
1,ENTP,"['INTJ', 'INTP', 'ENFP', 'INTJ', 'INTP', 'INTP..."
2,INTP,"['INTJ', 'INFP', 'INFP', 'INTP', 'INTP', 'INTJ..."
3,INTJ,"['INTJ', 'ISFJ', 'INFP', 'INTP', 'INTP', 'INTP..."
4,ENTJ,"['ENTJ', 'INTP', 'ENFP', 'INTP', 'ENTJ', 'INTJ..."


In [4]:
# 資料集切分為訓練集和驗證集
train_dialogues, val_dialogues, train_target, val_target = train_test_split(df['posts'], df['type'], test_size=0.1, random_state=45)

In [5]:
#資料集(驗證)重新排序(由0開始)
val_df = pd.DataFrame({'posts': val_dialogues, 'type': val_target})
val_df.reset_index(drop=True, inplace=True)

In [6]:
#再把排完之後的結果存回變數
val_dialogues = val_df['posts']
val_target = val_df['type']

In [7]:
pred_list = []
counter = 0
dimension_counts = {'E/I': 0,
                    'S/N': 0,
                    'T/F': 0,
                    'J/P': 0}
for index, one_list in enumerate(val_dialogues):
    one_list = eval(one_list)
    
    result = MBTI_prediction_by_3ways_jp.run(one_list)
    pred_list.append(result)
    
    if result == val_target[index]:
        counter += 1

    for n in range(4):
        if n == 0:
            if result[n] == val_target.iloc[index][n]:
                dimension_counts['E/I'] += 1
        elif n == 1:
            if result[n] == val_target.iloc[index][n]:
                dimension_counts['S/N'] += 1
        elif n == 2:
            if result[n] == val_target.iloc[index][n]:
                dimension_counts['T/F'] += 1
        elif n == 3:
            if result[n] == val_target.iloc[index][n]:
                dimension_counts['J/P'] += 1

Accuracy = counter/len(val_dialogues)
print(f"Accuracy: {Accuracy*100:.4f}%\n")

Accuracy: 43.6636%



In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 计算准确度
accuracy = accuracy_score(val_target, pred_list)

# 计算精确度
precision = precision_score(val_target, pred_list, average='weighted')

# 计算召回率
recall = recall_score(val_target, pred_list, average='weighted')

# 计算F1分数
f1 = f1_score(val_target, pred_list, average='weighted')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.44
Precision: 0.38
Recall: 0.44
F1 Score: 0.40


C:\Users\JenMing\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [9]:
EI_counts = dimension_counts['E/I']
SN_counts = dimension_counts['S/N']
TF_counts = dimension_counts['T/F']
JP_counts = dimension_counts['J/P']
print(f'E.I: {EI_counts}/{len(val_dialogues)} ')
print('Accuracy: '+ str(EI_counts/len(val_dialogues))+'\n')
print(f'S.N: {SN_counts}/{len(val_dialogues)} ')
print('Accuracy: '+ str(SN_counts/len(val_dialogues))+'\n')
print(f'T.F: {TF_counts}/{len(val_dialogues)} ')
print('Accuracy: '+ str(TF_counts/len(val_dialogues))+'\n')
print(f'J.P: {JP_counts}/{len(val_dialogues)} ')
print('Accuracy: '+ str(JP_counts/len(val_dialogues))+'\n')

E.I: 691/868 
Accuracy: 0.7960829493087558

S.N: 759/868 
Accuracy: 0.8744239631336406

T.F: 699/868 
Accuracy: 0.8052995391705069

J.P: 601/868 
Accuracy: 0.6923963133640553



In [10]:
with open("C:/Users/JenMing/Desktop/MBTI/3ways_result/note.txt", "w") as f:
    f.write(f"Accuracy: {Accuracy*100:.4f}%\n")
    f.write(f'E.I: {EI_counts}/{len(val_dialogues)} ')
    f.write('Accuracy: '+ str(EI_counts/len(val_dialogues))+'\n')
    f.write(f'S.N: {SN_counts}/{len(val_dialogues)} ')
    f.write('Accuracy: '+ str(SN_counts/len(val_dialogues))+'\n')
    f.write(f'T.F: {TF_counts}/{len(val_dialogues)} ')
    f.write('Accuracy: '+ str(TF_counts/len(val_dialogues))+'\n')
    f.write(f'J.P: {JP_counts}/{len(val_dialogues)} ')
    f.write('Accuracy: '+ str(JP_counts/len(val_dialogues))+'\n')
    f.write(f"Accuracy: {accuracy:.2f}\n")
    f.write(f"Precision: {precision:.2f}\n")
    f.write(f"Recall: {recall:.2f}\n")
    f.write(f"F1 Score: {f1:.2f}\n")